In [1]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

def fetch_data_in_bulk(model, fields, domain, uid):
    """Fetch data from Odoo in bulk for a specific model and domain."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert_products():
    print("Starting product data extraction...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch products
    print("Fetching product data...")
    products = fetch_data_in_bulk('product.product', ['id', 'display_name', 'standard_price', 'list_price', 'categ_id'], [], uid)

    if not products:
        print("No products fetched. Exiting.")
        return
    print(f"Fetched {len(products)} products.")

    # Step 3: Fetch product categories
    category_ids = list({product.get('categ_id', [])[0] for product in products if product.get('categ_id')})
    categories = fetch_data_in_bulk('product.category', ['id', 'name'], [['id', 'in', category_ids]], uid)

    # Map category IDs to names
    category_map = {category['id']: category['name'] for category in categories}

    # Step 4: Insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        # Create product table if it doesn't exist
        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'product'
        )
        BEGIN
            CREATE TABLE product (
                id INT PRIMARY KEY,
                name NVARCHAR(255),
                category NVARCHAR(255),
                cost DECIMAL(18, 2),
                sales_price DECIMAL(18, 2)
            );
        END
        """)
        conn.commit()

        print("Inserting product data into SQL Server...")
        for product in products:
            product_id = product.get('id')
            product_name = product.get('display_name')
            product_cost = product.get('standard_price')
            sales_price = product.get('list_price')

            # Get category name
            category_id = product.get('categ_id', [])[0] if product.get('categ_id') else None
            product_category = category_map.get(category_id, None)

            # Insert product if not exists
            cursor.execute("""
            IF NOT EXISTS (SELECT 1 FROM product WHERE id = %s)
            BEGIN
                INSERT INTO product (id, name, category, cost, sales_price) 
                VALUES (%s, %s, %s, %s, %s)
            END
            """, (product_id, product_id, product_name, product_category, product_cost, sales_price))
        
        conn.commit()
        print("Product data inserted successfully.")

        # Close SQL connection
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert_products()


Starting product data extraction...
Authenticated successfully. User ID: 112
Fetching product data...
Fetched 4459 products.
Inserting product data into SQL Server...
Product data inserted successfully.


In [3]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

def fetch_data_in_bulk(model, fields, domain, uid):
    """Fetch data from Odoo in bulk for a specific model and domain."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def get_full_category_path(category_id, category_map):
    """Recursively build full category path."""
    path = []
    while category_id:
        category = category_map.get(category_id)
        if not category:
            break
        path.append(category['name'])
        category_id = category['parent_id'][0] if category.get('parent_id') else None
    return " / ".join(reversed(path))

def process_and_insert_products():
    print("Starting product data extraction...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch products
    print("Fetching product data...")
    products = fetch_data_in_bulk('product.product', ['id', 'display_name', 'standard_price', 'list_price', 'categ_id'], [], uid)

    if not products:
        print("No products fetched. Exiting.")
        return
    print(f"Fetched {len(products)} products.")

    # Step 3: Fetch all product categories
    print("Fetching product categories...")
    category_ids = list({product.get('categ_id', [])[0] for product in products if product.get('categ_id')})
    categories = fetch_data_in_bulk('product.category', ['id', 'name', 'parent_id'], [['id', 'in', category_ids]], uid)

    # Create a dictionary for category lookup
    category_map = {category['id']: category for category in categories}

    # Step 4: Insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        # Create product table if it doesn't exist
        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'product'
        )
        BEGIN
            CREATE TABLE product (
                id INT PRIMARY KEY,
                name NVARCHAR(255),
                category NVARCHAR(1024),
                cost DECIMAL(18, 2),
                sales_price DECIMAL(18, 2)
            );
        END
        """)
        conn.commit()

        print("Inserting product data into SQL Server...")
        for product in products:
            product_id = product.get('id')
            product_name = product.get('display_name')
            product_cost = product.get('standard_price')
            sales_price = product.get('list_price')

            # Get full category path
            category_id = product.get('categ_id', [])[0] if product.get('categ_id') else None
            product_category = get_full_category_path(category_id, category_map) if category_id else None

            # Insert product if not exists
            cursor.execute("""
            IF NOT EXISTS (SELECT 1 FROM product WHERE id = %s)
            BEGIN
                INSERT INTO product (id, name, category, cost, sales_price) 
                VALUES (%s, %s, %s, %s, %s)
            END
            """, (product_id, product_id, product_name, product_category, product_cost, sales_price))
        
        conn.commit()
        print("Product data inserted successfully.")

        # Close SQL connection
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert_products()


Starting product data extraction...
Authenticated successfully. User ID: 112
Fetching product data...
Fetched 4459 products.
Fetching product categories...
Inserting product data into SQL Server...
Product data inserted successfully.


In [5]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

def fetch_data_in_bulk(model, fields, domain, uid):
    """Fetch data from Odoo in bulk for a specific model and domain."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def get_full_category_path(category_id, models, uid):
    """Recursively fetch and construct the full category hierarchy."""
    category_path = []
    while category_id:
        category = models.execute_kw(
            db_name,
            uid,
            password,
            'product.category',
            'read',
            [[category_id]],
            {'fields': ['id', 'name', 'parent_id']}
        )
        if not category:
            break
        category = category[0]
        category_path.append(category['name'])
        category_id = category['parent_id'][0] if category.get('parent_id') else None

    return " / ".join(reversed(category_path))  # Reverse to get root → leaf order

def process_and_insert_products():
    print("Starting product data extraction...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch products
    print("Fetching product data...")
    models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
    products = fetch_data_in_bulk('product.product', ['id', 'display_name', 'standard_price', 'list_price', 'categ_id'], [], uid)

    if not products:
        print("No products fetched. Exiting.")
        return
    print(f"Fetched {len(products)} products.")

    # Step 3: Insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        # Create product table if it doesn't exist
        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'product'
        )
        BEGIN
            CREATE TABLE product (
                id INT PRIMARY KEY,
                name NVARCHAR(255),
                category NVARCHAR(1024),
                cost DECIMAL(18, 2),
                sales_price DECIMAL(18, 2)
            );
        END
        """)
        conn.commit()

        print("Inserting product data into SQL Server...")
        for product in products:
            product_id = product.get('id')
            product_name = product.get('display_name')
            product_cost = product.get('standard_price')
            sales_price = product.get('list_price')

            # Get full category path
            category_id = product.get('categ_id', [])[0] if product.get('categ_id') else None
            product_category = get_full_category_path(category_id, models, uid) if category_id else None

            # Insert product if not exists
            cursor.execute("""
            IF NOT EXISTS (SELECT 1 FROM product WHERE id = %s)
            BEGIN
                INSERT INTO product (id, name, category, cost, sales_price) 
                VALUES (%s, %s, %s, %s, %s)
            END
            """, (product_id, product_id, product_name, product_category, product_cost, sales_price))
        
        conn.commit()
        print("Product data inserted successfully.")

        # Close SQL connection
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert_products()


Starting product data extraction...
Authenticated successfully. User ID: 112
Fetching product data...
Fetched 4459 products.
Inserting product data into SQL Server...
Error during SQL operations: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond


In [7]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

# Initialize batch size for SQL insert
batch_size = 100

def fetch_data_in_bulk(model, fields, uid):
    """Fetch data from Odoo in bulk for a specific model."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [[]],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def process_and_insert():
    print("Starting product data synchronization...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch product data
    print("Fetching product data...")
    products = fetch_data_in_bulk('product.product', ['id', 'display_name', 'standard_price', 'categ_id', 'list_price'], uid)

    if not products:
        print("No products fetched. Exiting.")
        return
    print(f"Fetched {len(products)} products.")

    # Step 3: Prepare and insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        # Create table if it doesn't exist
        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'product'
        )
        BEGIN
            CREATE TABLE product (
                id INT PRIMARY KEY,
                product_name NVARCHAR(255),
                product_cost DECIMAL(18, 2),
                product_category NVARCHAR(255),
                sales_price DECIMAL(18, 2)
            );
        END
        """)
        conn.commit()

        print("Inserting data into SQL Server...")
        batch_data = []

        for product in products:
            product_id = product.get('id')
            product_name = product.get('display_name')
            product_cost = product.get('standard_price')
            product_category = product.get('categ_id', [None])[0] if product.get('categ_id') else None
            sales_price = product.get('list_price')

            # Add to batch
            batch_data.append((product_id, product_name, product_cost, product_category, sales_price))

            # Insert in batches
            if len(batch_data) >= batch_size:
                cursor.executemany(f"""
                INSERT INTO product (id, product_name, product_cost, product_category, sales_price)
                VALUES (%s, %s, %s, %s, %s)
                """, batch_data)
                conn.commit()
                batch_data = []

        # Final batch insert
        if batch_data:
            cursor.executemany(f"""
            INSERT INTO product (id, product_name, product_cost, product_category, sales_price)
            VALUES (%s, %s, %s, %s, %s)
            """, batch_data)
            conn.commit()

        print("All product data inserted successfully.")

        # Close SQL connection
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert()


Starting product data synchronization...
Authenticated successfully. User ID: 112
Fetching product data...
Fetched 4459 products.
Inserting data into SQL Server...
All product data inserted successfully.


In [25]:
import xmlrpc.client
import pymssql

# Custom transport with a timeout
class TimeoutTransport(xmlrpc.client.Transport):
    def __init__(self, timeout=None):
        super().__init__()
        self.timeout = timeout

    def make_connection(self, host):
        connection = super().make_connection(host)
        connection.timeout = self.timeout
        return connection

# Odoo Connection Details
odoo_url = 'http://144.76.159.183:8069'
db_name = 'Backup_20250310'
username = 'admin'
password = 'admin'

# SQL Server Connection Details
server_name = 'SARAH\\SQLEXPRESS'
database_name = 'Odoo_sql_database'
sql_user_name = 'SuperAdmin'
sql_password = 'SuperAdmin'

# Initialize batch size for SQL insert
batch_size = 100

def fetch_data_in_bulk(model, fields, uid, domain=None):
    """Fetch data from Odoo in bulk for a specific model."""
    try:
        models = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/object', transport=TimeoutTransport(timeout=300))
        data = models.execute_kw(
            db_name,
            uid,
            password,
            model,
            'search_read',
            [domain] if domain else [[]],
            {'fields': fields}
        )
        return data
    except Exception as e:
        print(f"Error fetching data from Odoo for model {model}: {e}")
        return []

def get_category_hierarchy(category_id, category_map):
    """Recursively fetch the category hierarchy and return the full category path."""
    category_hierarchy = []
    current_id = category_id
    
    while current_id:
        # Ensure we're working with an integer ID
        if isinstance(current_id, (list, tuple)):
            current_id = current_id[0] if current_id else None
        
        if current_id in category_map:
            category = category_map[current_id]
            category_hierarchy.insert(0, category['name'])  # Add category to the front of the list
            current_id = category.get('parent_id')  # Move to the parent category
            if isinstance(current_id, (list, tuple)):
                current_id = current_id[0] if current_id else None
        else:
            break
            
    return " / ".join(category_hierarchy) if category_hierarchy else "Unknown"

def process_and_insert():
    print("Starting product data synchronization...")

    # Step 1: Authenticate with Odoo
    try:
        common = xmlrpc.client.ServerProxy(f'{odoo_url}/xmlrpc/2/common', transport=TimeoutTransport(timeout=300))
        uid = common.authenticate(db_name, username, password, {})
        if not uid:
            print("Failed to authenticate. Check your credentials.")
            return
        print(f"Authenticated successfully. User ID: {uid}")
    except Exception as e:
        print(f"Authentication error: {e}")
        return

    # Step 2: Fetch product data
    print("Fetching product data...")
    products = fetch_data_in_bulk('product.product', ['id', 'display_name', 'standard_price', 'categ_id', 'list_price'], uid)

    if not products:
        print("No products fetched. Exiting.")
        return
    print(f"Fetched {len(products)} products.")

    # Step 3: Fetch category data (including parent categories)
    print("Fetching product categories...")
    categories = fetch_data_in_bulk('product.category', ['id', 'name', 'parent_id'], uid)
    category_map = {cat['id']: cat for cat in categories}

    # Step 4: Prepare and insert data into SQL Server
    try:
        conn = pymssql.connect(
            server=server_name,
            user=sql_user_name,
            password=sql_password,
            database=database_name,
            charset='utf8'
        )
        cursor = conn.cursor()

        # Create table if it doesn't exist
        cursor.execute(f"""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES 
            WHERE TABLE_NAME = 'product'
        )
        BEGIN
            CREATE TABLE product (
                id INT PRIMARY KEY,
                product_name NVARCHAR(255),
                product_cost DECIMAL(18, 2),
                product_category NVARCHAR(1024),
                sales_price DECIMAL(18, 2)
            );
        END
        """)
        conn.commit()

        print("Inserting data into SQL Server...")
        batch_data = []

        for product in products:
            product_id = product.get('id')
            product_name = product.get('display_name')
            product_cost = product.get('standard_price', 0.0)
            
            # Handle categ_id which might be a list or None
            categ_id = product.get('categ_id')
            sales_price = product.get('list_price', 0.0)

            # Get the full category path using the category hierarchy function
            product_category = get_category_hierarchy(categ_id, category_map)

            # Add to batch
            batch_data.append((product_id, product_name, product_cost, product_category, sales_price))

            # Insert in batches
            if len(batch_data) >= batch_size:
                cursor.executemany(f"""
                INSERT INTO product (id, product_name, product_cost, product_category, sales_price)
                VALUES (%d, %s, %d, %s, %d)
                """, batch_data)
                conn.commit()
                batch_data = []

        # Final batch insert
        if batch_data:
            cursor.executemany(f"""
            INSERT INTO product (id, product_name, product_cost, product_category, sales_price)
            VALUES (%d, %s, %d, %s, %d)
            """, batch_data)
            conn.commit()

        print("All product data inserted successfully.")

        # Close SQL connection
        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error during SQL operations: {e}")

# Run the function
process_and_insert()

Starting product data synchronization...
Authenticated successfully. User ID: 112
Fetching product data...
Fetched 4459 products.
Fetching product categories...
Inserting data into SQL Server...
All product data inserted successfully.
